#### Motivo para o notebook existir

Será feita uma uma análise sobre exploratória, e alguns tratamentos de dados para uma base do Brasileirão encontrada no Kaggle. A exploração/transformação dos dados será em Spark, e o objetivo é aplicar conceitos básicos da ferramenta em algo descontraído/divertido.

Base de dados original:\
https://www.kaggle.com/datasets/ricardomattos05/brazilian-soccer-database?resource=download&select=Brazilian_Cup_Matches.csv


#### Campeonato Brasileiro de Futebol

O Campeonato Brasileiro de Futebol, também conhecido como Campeonato Brasileiro, Brasileirão e Série A, é a liga brasileira de futebol profissional entre clubes do Brasil, sendo a principal competição futebolística no país. É por meio dela que são indicados os representantes brasileiros para a Copa Libertadores da América (juntamente com o campeão da Copa do Brasil).

----

#### Importando as libs para começar a brincadeira


In [1]:
import os
os.environ["SPARK_HOME"] = "C:\Spark\spark-3.2.1-bin-hadoop2.7"

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession #Toda sessão de spark precisa conter uma spark session
from pyspark.sql.types import * #Para poder trabalhar nos tipos dos arquivos
from pyspark.sql import functions as f #importando todas as funções
from pyspark.sql.types import IntegerType

---

#### Criando uma Spark Session 


In [4]:
spark = SparkSession.builder.master('local[*]')\
    .appName("Análise brasileirão")\
    .getOrCreate()

spark

#### Carregando os dados

In [5]:
df = spark.read.csv("Brasileirao_Matches.csv", encoding='utf-8',\
    header=True,inferSchema=True)

#dispaly the data:
df.show(truncate=False)

+-------------------+----------------+---------------+----------------+---------------+---------+---------+------+-----+
|datetime           |home_team       |home_team_state|away_team       |away_team_state|home_goal|away_goal|season|round|
+-------------------+----------------+---------------+----------------+---------------+---------+---------+------+-----+
|2012-05-19 18:30:00|Palmeiras-SP    |SP             |Portuguesa-SP   |SP             |1        |1        |2012  |1    |
|2012-05-19 18:30:00|Sport-PE        |PE             |Flamengo-RJ     |RJ             |1        |1        |2012  |1    |
|2012-05-19 21:00:00|Figueirense-SC  |SC             |Nautico-PE      |PE             |2        |1        |2012  |1    |
|2012-05-20 16:00:00|Ponte Preta-SP  |SP             |Atletico-MG     |MG             |0        |1        |2012  |1    |
|2012-05-20 16:00:00|Corinthians-SP  |SP             |Fluminense-RJ   |RJ             |0        |1        |2012  |1    |
|2012-05-20 16:00:00|Botafogo-RJ

#### Limpeza dos dados

In [6]:
df.printSchema()

root
 |-- datetime: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- home_team_state: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- away_team_state: string (nullable = true)
 |-- home_goal: string (nullable = true)
 |-- away_goal: string (nullable = true)
 |-- season: integer (nullable = true)
 |-- round: integer (nullable = true)



In [7]:
#Descobrindo os valores nulos na base

df.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+--------+---------+---------------+---------+---------------+---------+---------+------+-----+
|datetime|home_team|home_team_state|away_team|away_team_state|home_goal|away_goal|season|round|
+--------+---------+---------------+---------+---------------+---------+---------+------+-----+
|       0|        0|              0|        0|              0|        0|        0|     0|    0|
+--------+---------+---------------+---------+---------------+---------+---------+------+-----+



In [25]:
#criando um hash
df = df.withColumn('hash_id', f.sha2(f.col('home_team'), 256))

Pontos que precisam de atenção:
- datetime está como string: Precisa ser tratado para date
- home_goal e away_goal:Estão como Strings, mas na são números inteiros

In [9]:
# Colocando cada tipo no seu lugar
brasileirao = df.withColumn("datetime", f.to_date(df.datetime)).\
    withColumn("home_goal", df.home_goal.cast(IntegerType())).\
    withColumn("away_goal", df.away_goal.cast(IntegerType()))

In [10]:
#Agora está tudo certo
brasileirao.printSchema()

root
 |-- datetime: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- home_team_state: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- away_team_state: string (nullable = true)
 |-- home_goal: integer (nullable = true)
 |-- away_goal: integer (nullable = true)
 |-- season: integer (nullable = true)
 |-- round: integer (nullable = true)
 |-- hash_id: string (nullable = true)



In [11]:
# função para tirar o prefixo de estado dos times:

brasileirao = brasileirao.withColumn(
    "home_team",
    f.expr("substring(home_team, 1, length(home_team) - 3)"))\
    .withColumn("away_team",
               f.expr("substring(away_team, 1, length(away_team) - 3)"))


In [12]:
brasileirao.show(5,truncate=False)

+----------+-----------+---------------+----------+---------------+---------+---------+------+-----+----------------------------------------------------------------+
|datetime  |home_team  |home_team_state|away_team |away_team_state|home_goal|away_goal|season|round|hash_id                                                         |
+----------+-----------+---------------+----------+---------------+---------+---------+------+-----+----------------------------------------------------------------+
|2012-05-19|Palmeiras  |SP             |Portuguesa|SP             |1        |1        |2012  |1    |962fe76aa393c46650935b55f592644752e860e9450824ae7623cd90de88e445|
|2012-05-19|Sport      |PE             |Flamengo  |RJ             |1        |1        |2012  |1    |1d9c7f9ec9a118384723171a17a6d7c6809751770cb2a03d88d4119fa94686c8|
|2012-05-19|Figueirense|SC             |Nautico   |PE             |2        |1        |2012  |1    |b4074aad67c39e35a2e689a3a3c3ca7e109500db50f99f5facf0c50352639676|
|201

In [14]:
#eu posso criar uma tabela dimensão com o nome dos times, id, e quantas vezes esse time foi campeão
hasht = brasileirao.withColumn('hash_id', f.sha2(f.col('home_team'), 256))

#### Criando uma nova

Será criada uma nova coluna que apresenta se o time que joja em casa perdeu, ganhou ou se foi um empate

In [15]:
brasileirao = brasileirao.withColumn(
        'results_thome',
        f.when(f.col("home_goal") > f.col("away_goal"), "ganhou")\
        .when(f.col("home_goal") < f.col("away_goal"), "perdeu")\
        .otherwise("empate"))\
        .withColumn("results_taway", 
                   f.when(f.col("away_goal") > f.col("home_goal"), "ganhou")\
                   .when(f.col("away_goal") < f.col("home_goal"), "perdeu")\
                   .otherwise("empate"))

In [24]:
brasileirao.select("datetime", "home_team", \
                   "away_team", "hash_id", "results_taway").show(5, truncate=True)

+----------+-----------+----------+--------------------+-------------+
|  datetime|  home_team| away_team|             hash_id|results_taway|
+----------+-----------+----------+--------------------+-------------+
|2012-05-19|  Palmeiras|Portuguesa|962fe76aa393c4665...|       empate|
|2012-05-19|      Sport|  Flamengo|1d9c7f9ec9a118384...|       empate|
|2012-05-19|Figueirense|   Nautico|b4074aad67c39e35a...|       perdeu|
|2012-05-20|Ponte Preta|  Atletico|7609b77126b835b3a...|       ganhou|
|2012-05-20|Corinthians|Fluminense|56e3228a60accfa0e...|       ganhou|
+----------+-----------+----------+--------------------+-------------+
only showing top 5 rows

